In [0]:
##Validamos si tenemos activo el SPARK
spark


In [0]:
##CONFIGURAR DATabricks
## No usamos Unit catalog, nousamos IAM
storage_account = "dletlsales20260117"
storage_key = "vF9xT8pNnPWgqBbTH0wJ4FjkORFnfOTZCd3lqowVFc+VxFNOgRipjP2mxeivS7LxaAHV0MLkxcs7+AStI6pLzw=="

spark.conf.set(
  f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
  storage_key
)


In [0]:
##Probamos conexion
dbutils.fs.ls("abfss://raw@dletlsales20260117.dfs.core.windows.net/")

[FileInfo(path='abfss://raw@dletlsales20260117.dfs.core.windows.net/errors/', name='errors/', size=0, modificationTime=1768670774000),
 FileInfo(path='abfss://raw@dletlsales20260117.dfs.core.windows.net/manual/', name='manual/', size=0, modificationTime=1768665730000),
 FileInfo(path='abfss://raw@dletlsales20260117.dfs.core.windows.net/sales/', name='sales/', size=0, modificationTime=1768676111000)]

In [0]:
##Conectamos databricks con ADLS GEN2
## Necesitamos dar permiso manage identity tal cual le dimos en al ADFS 
raw_path = "abfss://raw@dletlsales20260117.dfs.core.windows.net/"


In [0]:

from datetime import datetime
# Generar la fecha dinámica
fecha_actual = datetime.now().strftime("%Y-%m-%d")

##Leemos el CSV
df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .option("delimiter", ";")
    .csv(raw_path + f"sales/orders/ingestion_date={fecha_actual}/")
)

df_raw.display()


order_id,order_date,customer,product,amount
1,2026-01-15,Juan,Laptop,1200
2,2026-01-15,Ana,Mouse,25
3,2026-01-16,Luis,Teclado,80
4,2026-01-16,Carlos,Pad,80
5,2026-01-16,Fiorella,Laptop,1500
6,2026-01-16,Yenni,PC,3000
7,2026-01-16,Abigil,Telefono,850
8,2026-01-16,Javier,Telefono,850
9,2026-01-16,Romeo,Telefono,850
10,2026-01-16,Gabriel,Telefono,850


In [0]:
## Aplicamos validaciones 
from pyspark.sql.functions import col

df_raw.select(
    col("order_id").isNull().alias("order_id_null"),
    col("amount").isNull().alias("amount_null")
).display()



order_id_null,amount_null
false,false
false,false
false,false
false,false
false,false
false,false
false,false
false,false
false,false
false,false


In [0]:
### SILVER

##1. aplicamos transformaciones 
## eliminamos nulos, normalizamos texto, monto en decimales, fechas.
from pyspark.sql.functions import to_date, col, trim, current_date

df_silver = (
    df_raw
    # Tipos correctos
    .withColumn("order_id", col("order_id").cast("int"))
    .withColumn("order_date", to_date("order_date", "dd/MM/yyyy"))
    .withColumn("amount", col("amount").cast("decimal(10,2)"))

    # Normalización texto
    .withColumn("customer", trim(col("customer")))
    .withColumn("product", trim(col("product")))

    # Columnas técnicas
    .withColumn("ingestion_date", current_date())

    # Reglas de calidad
    .dropna(subset=["order_id", "order_date", "amount"])
)

df_silver.display()


order_id,order_date,customer,product,amount,ingestion_date
1,2026-01-15,Juan,Laptop,1200.00,2026-01-17
2,2026-01-15,Ana,Mouse,25.00,2026-01-17
3,2026-01-16,Luis,Teclado,80.00,2026-01-17
4,2026-01-16,Carlos,Pad,80.00,2026-01-17
5,2026-01-16,Fiorella,Laptop,1500.00,2026-01-17
6,2026-01-16,Yenni,PC,3000.00,2026-01-17
7,2026-01-16,Abigil,Telefono,850.00,2026-01-17
8,2026-01-16,Javier,Telefono,850.00,2026-01-17
9,2026-01-16,Romeo,Telefono,850.00,2026-01-17
10,2026-01-16,Gabriel,Telefono,850.00,2026-01-17


In [0]:
##Validacion
df_silver.printSchema()
df_silver.display()


root
 |-- order_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- customer: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: decimal(10,2) (nullable = true)
 |-- ingestion_date: date (nullable = false)



order_id,order_date,customer,product,amount,ingestion_date
1,2026-01-15,Juan,Laptop,1200.00,2026-01-17
2,2026-01-15,Ana,Mouse,25.00,2026-01-17
3,2026-01-16,Luis,Teclado,80.00,2026-01-17
4,2026-01-16,Carlos,Pad,80.00,2026-01-17
5,2026-01-16,Fiorella,Laptop,1500.00,2026-01-17
6,2026-01-16,Yenni,PC,3000.00,2026-01-17
7,2026-01-16,Abigil,Telefono,850.00,2026-01-17
8,2026-01-16,Javier,Telefono,850.00,2026-01-17
9,2026-01-16,Romeo,Telefono,850.00,2026-01-17
10,2026-01-16,Gabriel,Telefono,850.00,2026-01-17


In [0]:
## Guardamos en silver
silver_path = "abfss://silver@dletlsales20260117.dfs.core.windows.net/sales/orders"
(
    df_silver
    .write
    .format("delta")
    .mode("overwrite")   # Silver suele ser overwrite por partición lógica
    .option("overwriteSchema", "true")
    .save(silver_path)
)


**Creamos la base de datos Silver**

In [0]:

%sql
CREATE DATABASE IF NOT EXISTS silver;

## Creamos la tabla 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.sales_orders
USING DELTA
AS
SELECT *
FROM delta.`abfss://silver@dletlsales20260117.dfs.core.windows.net/sales/orders`;


num_affected_rows,num_inserted_rows


**Verificamos que exista la tabla**

In [0]:
%sql
SHOW TABLES IN silver;


database,tableName,isTemporary
silver,sales_orders,false


##Consultamos la tabla

In [0]:
%sql
SELECT *
FROM silver.sales_orders
LIMIT 10;



order_id,order_date,customer,product,amount,ingestion_date
1,2026-01-15,Juan,Laptop,1200.00,2026-01-17
2,2026-01-15,Ana,Mouse,25.00,2026-01-17
3,2026-01-16,Luis,Teclado,80.00,2026-01-17
4,2026-01-16,Carlos,Pad,80.00,2026-01-17
5,2026-01-16,Fiorella,Laptop,1500.00,2026-01-17
6,2026-01-16,Yenni,PC,3000.00,2026-01-17
7,2026-01-16,Abigil,Telefono,850.00,2026-01-17
8,2026-01-16,Javier,Telefono,850.00,2026-01-17
9,2026-01-16,Romeo,Telefono,850.00,2026-01-17
10,2026-01-16,Gabriel,Telefono,850.00,2026-01-17


## CAPA GOLD 
1: Creamos la base de datos gold

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold;


**Query de ventas diarias**

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.daily_sales
USING DELTA
AS
SELECT
    order_date,
    COUNT(order_id)      AS total_orders,
    SUM(amount)          AS total_sales
FROM silver.sales_orders
GROUP BY order_date;


num_affected_rows,num_inserted_rows


Validamos


In [0]:
%sql
SELECT * 
FROM gold.daily_sales
ORDER BY order_date;


order_date,total_orders,total_sales
2026-01-15,2,1225.00
2026-01-16,8,8060.00


##GOLD: Ventas por producto

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.product_sales
USING DELTA
AS
SELECT
    product,
    COUNT(order_id) AS total_orders,
    SUM(amount)     AS total_sales
FROM silver.sales_orders
GROUP BY product;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.product_sales

product,total_orders,total_sales
PC,1,3000.00
Telefono,4,3400.00
Pad,1,80.00
Laptop,2,2700.00
Mouse,1,25.00
Teclado,1,80.00


## GOLD VENTA POR CLIENTES

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.customer_sales
USING DELTA
AS
SELECT
    customer,
    COUNT(order_id) AS total_orders,
    SUM(amount)     AS total_sales
FROM silver.sales_orders
GROUP BY customer;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.customer_sales

customer,total_orders,total_sales
Abigil,1,850.00
Carlos,1,80.00
Gabriel,1,850.00
Ana,1,25.00
Fiorella,1,1500.00
Javier,1,850.00
Luis,1,80.00
Juan,1,1200.00
Yenni,1,3000.00
Romeo,1,850.00
